## Investigation of Error Sources in a Cohort Selection Service Based on a Multimodal Large Language Models


In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 8.9 MB/s eta 0:00:00


### Import packages

In [ ]:
import pathlib
import textwrap
import PIL.Image
import google.generativeai as genai
import pandas as pd
from IPython.display import display
from IPython.display import Markdown
import os
from tqdm import tqdm
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

from google.generativeai.types import HarmCategory, HarmBlockThreshold


In [ ]:
from google.colab import drive # подключаем google диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Used to securely store your API key
from google.colab import userdata

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## List models
Ограничение скорости для моделей Gemini-Pro составляет 60 запросов в минуту (RPM).

## Generate text from image and text inputs

Gemini принимает как текст, так и изображения на вход. API GenerativeModel.generate_content предназначен для обработки мультимодальных запросов и возвращает текстовый вывод.

In [ ]:
crit1 = PIL.Image.open('/content/drive/MyDrive/диплом/test/crit1.png') #критерии отбора пациентов
crit2 = PIL.Image.open('/content/drive/MyDrive/диплом/test/crit2.png')
crit3 = PIL.Image.open('/content/drive/MyDrive/диплом/test/crit3.png')

In [ ]:
hw_img1 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo1.jpg') # рукописная карта пациента
hw_img2 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo2.jpg')
hw_img3 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo3.jpg')
hw_img4 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo4.jpg')
hw_img5 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo5.jpg')
hw_img6 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo6.jpg')
hw_img7 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo7.jpg')
hw_img8 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo8.jpg')
hw_img9 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo9.jpg')
hw_img10 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo10.jpg')
hw_img11 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo11.jpg')
hw_img12 = PIL.Image.open('/content/drive/MyDrive/диплом/test/photo/photo12.jpg')

In [ ]:
img1 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test1.png') # карта пациента
img2 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test2.png')
img3 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test3.png')
img4 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test4.png')
img5 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test5.png')
img6 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test6.png')
img7 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test7.png')
img8 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test8.png')
img9 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test9.png')
img10 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test10.png')
img11 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test11.png')
img12 = PIL.Image.open('/content/drive/MyDrive/диплом/test/text/test12.png')

In [ ]:
#model = genai.GenerativeModel('gemini-pro-vision', generation_config={"temperature": 0.0})    #gemini-pro-vision

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest', generation_config={"temperature": 0.0})  #gemini-1.5-pro-latest

In [ ]:
response = model.generate_content(img1)            # подаем в модель карту пациента, без промпта. Она сразу дает рекомендации
to_markdown(response.text)

In [ ]:
text_crit1 = '''
Clinical Trial Criteria: {
Inclusion Criteria:
Subjects must have smoked on average of at least ten cigarettes per day during the past year;
Patient weighs more than 220 pounds.

Exclusion Criteria:
Subjects with any history of clinically significant cardiovascular disease;
Uncontrolled hypertension. }'''  # текстовое описание критерия

In [ ]:
text_crit2 = '''
Clinical Trial Criteria: {
Inclusion Criteria:
Required:
At least 18 years old;
Documented HIV infection.

Exclusion Criteria:
Active infectious diseases, except HIV;
Diabetes prior to starting HIV medications;
Alcohol or drug abuse;
Cancer or lymphoma. }'''

In [ ]:
text_crit3 = '''Clinical Trial Criteria:{
Inclusion criteria:
Age 16 to 59 years;
Infectious inflammatory diseases of throat and ears (tonsillitis, pharyngitis, otitis, etc.) recurring more than 3 times per year;
No history of allergic reactions.

Exclusion criteria:
Past surgical interventions on ENT organs;
Presence of other infectious diseases (tuberculosis, hepatitis, HIV, etc.) }'''

In [ ]:
prompt_zero_shot = '''This is the patient selection criterion for a medical study.
Next, I will show you patient cards and ask if this patient fits the criteria for this study,
and you should answer: yes or no'''  #промпт yes/no  (zero shot)

In [ ]:
prompt_few_shot = '''This is the criterion for selecting patients for medical research.
Next, I will show you the medical records of the patients and ask if this patient meets the criteria for this study. This is necessary to make a decision:
YES, the patient is suitable or NO, the patient is not suitable.
For example,
Clinical trial criteria:{
Inclusion criteria:
- Patients should have smoked an average of at least ten cigarettes a day over the past year
- The patient's weight exceeded 220 pounds
Exclusion criteria:
- Patients with a history of clinically significant cardiovascular diseases
- Uncontrolled arterial hypertension}
Patient's chart: A 44-year-old man weighing 250 pounds complains of severe pain in his left heel.
He says he has been experiencing this pain for about two weeks now.
The patient appears to have a pigmentation disorder on the arms, legs and other parts of the body, including the face, which may or may not change over time.
He has erythematous shingles in the form of moccasins on the plantar surfaces of both feet.  He has redness of the 2nd, 3rd and 4th toes of his right foot.
Answer: there is no evidence that the patient smokes, so he does not meet the inclusion criteria. Conclusion - no, the patient is not suitable.''' # промпт few-shot

In [ ]:
prompt_CoT = '''This is the criterion for selecting patients for medical research.
Next, I will show you the patient cards and ask if this patient meets the criteria for this study.
Think and answer step by step, making up a logical chain of reasoning, the result of which should be the answer:
yes - the patient is suitable for research or no - the patient is not suitable for research.''' #промпт CoT

In [ ]:
prompt_ToT = '''This is the criterion for selecting patients for medical research.
Next, I will show you the patient cards and ask if this patient meets the criteria for this study.
Imagine that three experts in the field answer a question.
All the experts take turns writing down the first step of their reasoning,
then discussing it with the group.
Then they move on to the second step of reasoning and so on.
If some expert realized that he was wrong, then he leaves. As a result, experts should give a conclusion:
yes, the patient is suitable for research or not - the patient is not suitable for research.'''

## Попросим модель прочитать все рукописные изображения и проанализируем наличие ошибок

In [ ]:
read_1 = model.generate_content(['read the patient card and retell it', hw_img1])   # рукописный текст
read_1.resolve()
to_markdown(read_1.text)

>  Patient is a 55-year-old male who presents with complaints of shortness of breath with exertion, morning cough, and periodic leg swelling. He has a history of smoking an average of 15 cigarettes per day for the past 20 years. He weighs 285 lbs at 5'11" (BMI = 38.5 kg/m2) and leads a sedentary lifestyle. He drinks alcohol occasionally. He was diagnosed with asthma in 2002 and receives maintenance bronchodilator therapy. His blood pressure is 160-165 mm Hg systolic and 100 mm Hg diastolic. He has a past medical history of obesity since 2000.

1.5  
The patient is a 55-year-old man. His chief complaints are shortness of breath with exertion, morning cough, and periodic leg swelling.

His history includes smoking an average of 15 cigarettes per day for the past 30 years. He weighs 225 lbs at 5'11" (BMI = 38.5 kg/m2). He leads a sedentary lifestyle and drinks alcohol occasionally.

He was diagnosed with asthma in 2002 and receives maintenance bronchodilator therapy. His blood pressure is systolic 160-165 mmHg and diastolic 110-110 mmHg.

His past medical history includes Class 2-3 obesity since 2000.  
1.0  
Patient is a 55-year-old male who presents with complaints of shortness of breath with exertion, morning cough, and periodic leg swelling. He has a history of smoking an average of 15 cigarettes per day for the past 20 years. He weighs 285 lbs at 5'11" (BMI = 38.5 kg/m2) and leads a sedentary lifestyle. He drinks alcohol occasionally. He was diagnosed with asthma in 2002 and receives maintenance bronchodilator therapy. His blood pressure is 160-165 mm Hg systolic and 100 mm Hg diastolic. He has a past medical history of obesity since 2000.

In [ ]:
read_1t = model.generate_content(['read the patient card and retell it', img1])
read_1t.resolve()
to_markdown(read_1t.text)

>  A 55-year-old male complains of **shortness of breath** with exertion and **morning cough**. He has a **history of asthma** and is on maintenance budesonide therapy. His **blood pressure is 160-165/100-110 mm Hg**. He is **obese** (BMI = 38.5 kg/m2) and has **class 2-3 obesity** since 2000. He also has **periodic leg swelling**.

1.5  
This is a 55-year-old male patient presenting with shortness of breath upon exertion and a morning cough. He also experiences periodic leg swelling. He has a history of smoking 15 cigarettes per day for the past 20 years. He weighs 275 lbs and is 5'11" tall, giving him a BMI of 38.5 kg/m2. He leads a sedentary lifestyle and drinks alcohol occasionally. He was diagnosed with asthma in 2002 and receives maintenance budesonide therapy. His blood pressure is 160-165 mmHg systolic and 100-110 mmHg diastolic. His past medical history includes Class 2-3 obesity since 2000.  
1.0  
A 55-year-old male complains of shortness of breath with exertion and morning cough. He has a history of asthma and is on maintenance budesonide therapy. His blood pressure is 160-165/100-110 mm Hg. He is obese (BMI = 38.5 kg/m2) and has class 2-3 obesity since 2000. He also has periodic leg swelling.

In [ ]:
read_2 = model.generate_content(['read the patient card and retell it', hw_img2])  # рукописный текст
read_2.resolve()
to_markdown(read_2.text)

>  A 38-year-old female accountant works in a small company. Due to work, she leads a sedentary lifestyle. She often suffers from tonsillitis and the flu. Her mother struggled with heart disease throughout her life and died of a heart attack at the age of 52. At the age of 37, she was diagnosed with HIV. Since then, she has been taking Atripla regularly. After starting treatment, she developed weakness and fatigue. She used to smoke a lot but gave up the habit a year ago.

1.5  
A 38-year-old woman works as an accountant in a small company. Due to work, she leads a sedentary lifestyle. She often suffers from tonsillitis and flu. Her mother struggled with heart disease throughout her life and died of a heart attack at age 52. At the age of 37, she was diagnosed with HIV. Since then, she has been taking Atripla regularly. After starting treatment, she developed weakness and fatigue. She used to smoke a lot, but gave up the habit a year ago.  
1.0  
A 38-year-old female accountant works in a small company. Due to work, she leads a sedentary lifestyle. She often suffers from tonsillitis and the flu. Her mother struggled with heart disease throughout her life and died of a heart attack at the age of 52. At the age of 37, she was diagnosed with HIV. Since then, she has been taking Atripla regularly. After starting treatment, she developed weakness and fatigue. She used to smoke a lot but gave up the habit a year ago.

In [ ]:
read_2t = model.generate_content(['read the patient card and retell it', img2])
read_2t.resolve()
to_markdown(read_2t.text)

>  A 38-year-old female accountant presents with complaints of fatigue and weakness. She has a history of frequent tonsillitis and influenza, as well as a family history of heart disease. Her mother died of a heart attack at age 52. The patient was diagnosed with HIV at age 37 and has been taking Atripla since then. She used to smoke but quit a year ago.

1.5  
This is a 38-year-old woman who works as an accountant and leads a sedentary lifestyle. She frequently experiences tonsillitis and the flu. Her mother had a history of heart disease and passed away from a heart attack at the age of 52. At 37, the patient was diagnosed with HIV and has been taking Atripla regularly since then. After starting treatment, she developed weakness and fatigue. She used to be a heavy smoker but quit a year ago.  
1.0  
A 38-year-old female accountant presents with complaints of fatigue and weakness. She has a history of frequent tonsillitis and influenza, as well as a family history of heart disease. Her mother died of a heart attack at age 52. The patient was diagnosed with HIV at age 37 and has been taking Atripla since then. She used to smoke but quit a year ago.

In [ ]:
read_3 = model.generate_content(['read the patient card and retell it', hw_img3]) # рукописный текст
read_3.resolve()
to_markdown(read_3.text)

>  A 56-year-old man works as an assistant to the captain on a long-haul cargo ship. Due to the specifics of his job, he suffers from chronic inflammation of the middle ear and complains of poor hearing. In his medical history, he has had rheumatoid arthritis for the last 6 years. he says that he periodically takes Prednisone and Arava. According to the patient, the rheumatologist did not want him to take long-term medication, so he would give him Prednisone, then switch to Arava, and then switch him back to Prednisone again. he reports having taken Prednisone for the last 6-9 months. At the time of admission, the patient's blood pressure is 135/80 mmHg, height is 193 cm, weight is 130 kg, and he does not consume alcohol. He smokes up to 20 cigarettes a day.

1.5  
A 56-year-old man works as an assistant to the captain on a long-haul cargo ship. Due to the specifics of his job, he suffers from chronic inflammation of the middle ear and complains of poor hearing. In his medical history, he has had rheumatoid arthritis for the last 6 years. He says that he periodically takes Prednisone and Arava. According to the patient, the rheumatologist did not want him to take long-term medication, so he would give him Prednisone, then switch to Arava, and then switch him back to Prednisone again. He reports having taken Prednisone for the last 6-9 months. At the time of admission, the patient's blood pressure is 135/80 mm Hg, height is 193 cm, weight is 130 kg, and he does not consume alcohol. He smokes up to 20 cigarettes a day.  
1.0  
A 56-year-old man works as an assistant to the captain on a long-haul cargo ship. Due to the specifics of his job, he suffers from chronic inflammation of the middle ear and complains of poor hearing. In his medical history, he has had rheumatoid arthritis for the last 6 years. he says that he periodically takes Prednisone and Arava. According to the patient, the rheumatologist did not want him to take long-term medication, so he would give him Prednisone, then switch to Arava, and then switch him back to Prednisone again. he reports having taken Prednisone for the last 6-9 months. At the time of admission, the patient's blood pressure is 135/80 mmHg, height is 193 cm, weight is 130 kg, and he does not consume alcohol. He smokes up to 20 cigarettes a day.

In [ ]:
read_3t = model.generate_content(['read the patient card and retell it', img3])
read_3t.resolve()
to_markdown(read_3t.text)

>  A 56-year-old man works as an assistant to the captain on a long-haul cargo ship. Due to the specifics of his job, he suffers from chronic inflammations of the middle ear and complains of poor hearing. In his medical history, he has had rheumatoid arthritis for the last 6 years. He says that he periodically takes Prednisone and Arava. According to the patient, the rheumatologist did not want him to take long-term medication, so he would give him Prednisone, then switch to Arava, and then switch him back to Prednisone again. He reports having taken Prednisone for the last 6-9 months. At the time of admission, the patient's blood pressure is 125/80 mmHg. Height is 193 cm, weight is 130 kg, and he does not consume alcohol. He smokes up to 20 cigarettes a day.

1.5  
The patient is a 56-year-old man who works as an assistant to the captain on a long-haul cargo ship. His job has caused chronic middle ear inflammation, leading to poor hearing. He has a 6-year history of rheumatoid arthritis and takes Prednisone and Arava periodically. His rheumatologist prefers to avoid long-term medication, so he cycles the patient between Prednisone and Arava. The patient has been on Prednisone for the past 6-9 months.

At the time of admission, his blood pressure is 125/80 mmHg, height is 193 cm, and weight is 130 kg. He does not consume alcohol but smokes up to 20 cigarettes per day.  
1.0  
A 56-year-old man works as an assistant to the captain on a long-haul cargo ship. Due to the specifics of his job, he suffers from chronic inflammations of the middle ear and complains of poor hearing. In his medical history, he has had rheumatoid arthritis for the last 6 years. He says that he periodically takes Prednisone and Arava. According to the patient, the rheumatologist did not want him to take long-term medication, so he would give him Prednisone, then switch to Arava, and then switch him back to Prednisone again. He reports having taken Prednisone for the last 6-9 months. At the time of admission, the patient's blood pressure is 125/80 mmHg. Height is 193 cm, weight is 130 kg, and he does not consume alcohol. He smokes up to 20 cigarettes a day.

In [ ]:
read_4 = model.generate_content(['read the patient card and retell it', hw_img4])  # рукописный текст
read_4.resolve()
to_markdown(read_4.text)

>  A 23-year-old male complains of periodic pain in his throat and ears. He has a long history of frequent streptococcal pharyngitis and acute otitis media infections occurring more than three times per year since he was a teenager. He was treated with several courses of antibiotics over the years. Three years ago he had anaphylactic shock from penicillin antibiotics.

1.5  
This is a patient card for a 23-year-old male. His main complaint is periodic pain in the throat.

His history of present illness indicates he has a long history of frequent strep throat/pharyngitis and acute otitis media infections, occurring more than 3 times per year since he was a teenager. He was treated with several courses of antibiotics over the years. Three years ago, he had anaphylactic shock from penicillin antibiotics  
1.0  
A 23-year-old male complains of periodic pain in his throat and ears. He has a long history of frequent streptococcal pharyngitis and acute otitis media infections occurring more than three times per year since he was a teenager. He was treated with several courses of antibiotics over the years. Three years ago he had anaphylactic shock from penicillin antibiotics.

In [ ]:
read_4t = model.generate_content(['read the patient card and retell it', img4])
read_4t.resolve()
to_markdown(read_4t.text)

>  A 23-year-old male complains of periodic pain in his throat and ears. He has a long history of frequent streptococcal pharyngitis and acute otitis media infections, occurring more than 3 times per year since he was a teenager. He was treated with several courses of antibiotics over the years. Three years ago, he had anaphylactic shock from penicillin antibiotics.

1.5  
This is a 23-year-old male patient presenting with periodic pain in his throat and ears. He has a long history of frequent strep throat and ear infections, occurring more than three times per year since he was a teenager. He has been treated with multiple courses of antibiotics over the years. Three years ago, he experienced anaphylactic shock from penicillin antibiotics.  
1.0  
A 23-year-old male complains of periodic pain in his throat and ears. He has a long history of frequent streptococcal pharyngitis and acute otitis media infections, occurring more than 3 times per year since he was a teenager. He was treated with several courses of antibiotics over the years. Three years ago, he had anaphylactic shock from penicillin antibiotics.

In [ ]:
read_5 = model.generate_content(['read the patient card and retell it', hw_img5])  # рукописный текст
read_5.resolve()
to_markdown(read_5.text)

>  The patient is a 42-year-old man who was diagnosed with HIV in 2019. He was started on antiretroviral therapy and had an undetectable viral load. In 2020, he developed significant fat depletion after starting HIV treatment. He also has low leptin levels in his blood and fasting triglycerides of 420 mg/dL. He has a history of lung cancer (non-small cell carcinoma) in 2010, but is currently in remission. He undergoes cancer screening every 6 months. His father passed away from lung cancer. The patient has no chronic infections apart from HIV and does not abuse alcohol or substances.

1.5  
This is a patient card for a 62-year-old man. He was diagnosed with HIV in 2019. While on antiretroviral therapy, he developed a significant fat depletion after starting HIV treatment. He has low leptin levels in his blood and fasting triglycerides of 920 mg/dL.

He has a history of lung cancer (adenocarcinoma) in 2010 and is currently in remission. He undergoes cancer screening every 6 months, and so far, the cancer has passed away from his lung.

He has no chronic infections apart from HIV. He does not consume alcohol or abuse any substances.  
1.0  
The patient is a 42-year-old man who was diagnosed with HIV in 2019. He was started on antiretroviral therapy and had an undetectable viral load. In 2020, he developed significant fat depletion after starting HIV treatment. He also has low leptin levels in his blood and fasting triglycerides of 420 mg/dL. He has a history of lung cancer (non-small cell carcinoma) in 2010, but is currently in remission. He undergoes cancer screening every 6 months. His father passed away from lung cancer. The patient has no chronic infections apart from HIV and does not abuse alcohol or substances.

In [ ]:
read_5t = model.generate_content(['read the patient card and retell it', img5])
read_5t.resolve()
to_markdown(read_5t.text)

>  A 42-year-old male was diagnosed with HIV in 2019 and has been on antiretroviral therapy since then. He has a history of lung cancer (adenocarcinoma) in 2010 and is currently in remission. He undergoes cancer screening every 6 months. His father passed away from lung cancer. He has no chronic infections apart from HIV and no history of alcohol or substance abuse.
> 
> The patient developed significant fat depletion after starting HIV treatment and has low leptin levels in his blood. His fasting triglycerides are 420 mg/dL.

1.5  
This is a 42-year-old male patient who was diagnosed with HIV in 2019 and is currently on antiretroviral therapy. He experienced significant fat loss after starting HIV treatment and has low leptin levels in his blood. His fasting triglycerides are elevated at 420 mg/dL.

He has a history of lung cancer (adenocarcinoma) diagnosed in 2010, which is currently in remission. He undergoes cancer screening every six months. His father passed away from lung cancer.

The patient has no chronic infections other than HIV and does not have a history of alcohol or substance abuse.  
1.0  
A 42-year-old male was diagnosed with HIV in 2019 and has been on antiretroviral therapy since then. He has a history of lung cancer (adenocarcinoma) in 2010 and is currently in remission. He undergoes cancer screening every 6 months. His father passed away from lung cancer. He has no chronic infections apart from HIV and no history of alcohol or substance abuse.

The patient developed significant fat depletion after starting HIV treatment and has low leptin levels in his blood. His fasting triglycerides are 420 mg/dL.

In [ ]:
read_6 = model.generate_content(['read the patient card and retell it', hw_img6])  # рукописный текст
read_6.resolve()
to_markdown(read_6.text)

>  The patient is a 31-year-old male who complains of a sore throat and fever up to 38°C. He has a history of recurrent episodes of acute tonsillitis (4-5 times a year) over the past 5 years. He denies any allergic reactions. He is a heavy smoker (smokes a pack a day).

1.5  
The patient is a 31-year-old who presents with a sore throat and fever up to 38°C.

Over the past 5 years, they have had recurrent episodes of acute tonsillitis, occurring 4-5 times a year.

They deny any allergic reactions.

The patient is a heavy smoker, smoking a pack a day.  
1.0  
The patient is a 31-year-old male who complains of a sore throat and fever up to 38°C. He has a history of recurrent episodes of acute tonsillitis (4-5 times a year) over the past 5 years. He denies any allergic reactions. He is a heavy smoker (smokes a pack a day).

In [ ]:
read_6t = model.generate_content(['read the patient card and retell it', img6])
read_6t.resolve()
to_markdown(read_6t.text)

>  The patient is a 31-year-old male who is a heavy smoker (one pack per day) and has a history of recurrent episodes of acute tonsillitis (4-5 times per year). He denies any allergic reactions. He presents with a sore throat and a fever of 38°C.

1.5  
The patient is a 31-year-old who presents with a sore throat and fever up to 38°C. They have a history of recurrent episodes of acute tonsillitis, experiencing it 4-5 times per year for the past 5 years. They deny any allergic reactions. The patient is also a heavy smoker, smoking a pack of cigarettes a day.  
1.0  
The patient is a 31-year-old male who is a heavy smoker (one pack per day) and has a history of recurrent episodes of acute tonsillitis (4-5 times per year). He denies any allergic reactions. He presents with a sore throat and a fever of 38°C.

In [ ]:
read_7 = model.generate_content(['read the patient card and retell it', hw_img7])  # рукописный текст
read_7.resolve()
to_markdown(read_7.text)

>  A 45-year-old white male with a history of schizophrenia and AIDS was admitted for disorganized and assaultive behaviors while off all medications for the last six months. Psych consult- Schizophrenia. IDENTIFYING DATA CHIEF COMPLAINT: "I'm in jail because I was involuntarily committed. I'm claiming a 4th-hour Involudntary treatment for grave disability. History of present illness: The patient has minimal insight into the circumstances that resulted in his admission. The patient reports being diagnosed with HIV and AIDS in 1994. Current medications: none. Allergies: no known drug allergies. Social history: The patient lives with his partner. He is unemployed. Details of his educational and occupational history are not currently known. His source of finances is also unknown, though social security disability is presumed. Substance and alcohol history: The patient smoked one to two packs per day for most of the last year. At the time of admission, the patient's blood pressure is 130/85 mmHg, height is 169.0cm, weight is 109.1 kg.

1.5  
A 55-year-old white male with a history of schizophrenia and AIDS was admitted for disorganized and assaultive behaviors while off all medications for the last six months. Psych consult - schizophrenia. Identifying data, chief complaint: "I'm in jail because I was wrongly arrested." The patient is admitted on a 72-hour involuntary treatment act for grave disability. History of present illness: The patient has minimal insight into the circumstances that resulted in this admission. The patient reports being diagnosed with HIV and AIDS in 1994. Current medications: none; allergies: no known drug allergies. Social and developmental history: The patient lives with his partner. He is unemployed. Details of his educational and occupational history are not currently known. His source of finances is also unknown, though social security disability is presumed. Substance and alcohol history: The patient smoked one to two packs per day for most of the last year. At the time of admission, the patient's blood pressure is 130/85 mmHg, height is 169 cm, weight 104 kg.  
1.0  
A 45-year-old white male with a history of schizophrenia and AIDS was admitted for disorganized and assaultive behaviors while off all medications for the last six months. Psych consult- Schizophrenia. IDENTIFYING DATA CHIEF COMPLAINT: "I'm in jail because I was involuntarily committed. I'm claiming a 4th-hour Involudntary treatment for grave disability. History of present illness: The patient has minimal insight into the circumstances that resulted in his admission. The patient reports being diagnosed with HIV and AIDS in 1994. Current medications: none. Allergies: no known drug allergies. Social history: The patient lives with his partner. He is unemployed. Details of his educational and occupational history are not currently known. His source of finances is also unknown, though social security disability is presumed. Substance and alcohol history: The patient smoked one to two packs per day for most of the last year. At the time of admission, the patient's blood pressure is 130/85 mmHg, height is 169.0cm, weight is 109.1 kg.

In [ ]:
read_7t = model.generate_content(['read the patient card and retell it', img7])
read_7t.resolve()
to_markdown(read_7t.text)

>  The patient is a 45-year-old white male with a history of schizophrenia and AIDS. He was admitted for disorganized and assaultive behaviors while off all medications for the last six months. He was arrested and is currently in jail. The patient has minimal insight into the circumstances that resulted in his admission. He reports being diagnosed with HIV and AIDS in 1994. He is unemployed and his source of finances is unknown. He smoked one to two packs of cigarettes per day for most of the last year. His blood pressure is 130/85 mmHg and his height is 169 cm and weight is 101 kg.

1.5  
The patient is a 45-year-old white male with a history of schizophrenia and AIDS. He was admitted for disorganized and assaultive behaviors after being off all medications for the past six months. He claims he was wrongly arrested and is currently admitted on a 72-hour involuntary treatment act for grave disability. He has minimal insight into the circumstances that resulted in his admission but reports being diagnosed with HIV and AIDS in 1994. He is not currently on any medications and has no known drug allergies. He lives with his partner and is unemployed. Details of his educational and occupational history are unknown, though social security disability is presumed. He smoked one to two packs of cigarettes per day for most of the last year. At the time of admission, his blood pressure was 130/85 mmHg, height is 169 cm, and weight is 101 kg.  
1.0  
The patient is a 45-year-old white male with a history of schizophrenia and AIDS. He was admitted for disorganized and assaultive behaviors while off all medications for the last six months. He was arrested and is currently in jail. The patient has minimal insight into the circumstances that resulted in his admission. He reports being diagnosed with HIV and AIDS in 1994. He is unemployed and his source of finances is unknown. He smoked one to two packs of cigarettes per day for most of the last year. His blood pressure is 130/85 mmHg and his height is 169 cm and weight is 101 kg.

In [ ]:
read_8 = model.generate_content(['read the patient card and retell it', hw_img8])  # рукописный текст
read_8.resolve()
to_markdown(read_8.text)

ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.

1.5  
error  
1.0  
error

In [ ]:
read_8t = model.generate_content(['read the patient card and retell it', img8])
read_8t.resolve()
to_markdown(read_8t.text)

ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.

1.5  
A 44-year-old, 250-pound male patient is experiencing extreme pain in his left heel, which is his primary complaint. This pain has been present for approximately two weeks. He works on concrete floors and reports that the pain is particularly intense in the mornings upon waking or after periods of sitting, making walking very difficult.

The patient also presents with a macular blotching of the skin on his arms, face, legs, feet, and the rest of his body. He describes this as a pigmentation disorder that he has had since he was 17 years old. Additionally, he has redness and signs of infection in his right toes.

Further examination reveals a possible pigmentation disorder, which may or may not change over time, on his arms, legs, and other parts of his body, including his face. He also exhibits an erythematous, moccasin-pattern tinea pedis on the plantar surfaces of both feet, along with redness in toes 2, 3, and 4 of his right foot. Direct pressure on the plantar aspect of his left heel elicits extreme, exquisite pain.  
1.0  
error

In [ ]:
read_9 = model.generate_content(['read the patient card and retell it', hw_img9])  # рукописный текст
read_9.resolve()
to_markdown(read_9.text)

>  The patient is a 49-year-old white female. Her weight is 221.5 pounds, blood pressure 128/74, pulse 80, and respirations 16. On examination, this is a flaring of her acne with small folliculitis bid lesions. The patient has been taking amoxicillin 500mg qid, and using Tazorac cream q1, and her face is doing well. She has been out of her medicine now for three days also. She has also been getting photofacials at Healing Waters and was wondering about what we could offer as far as cosmetic procedures and skin care products, etc. The patient is married. She is a secretary. FAMILY, SOCIAL, AND ALLERGY HISTORY: She has hay fever, eczema, sinus, and hives. She has no skin cancers or psoriasis. Her mother had oral cancer. She smokes on average one pack a day. No blood tests. Had some sunburn in the past. She is on benozoyl peroxide and aspirin, vitamins, Lexapro, Edecxor, Ditropan.

1.5  
The patient is a 49-year-old white female. She weighs 221.5 pounds, her blood pressure is 128/79, her pulse is 80, and her respirations are 16. On examination, she has a flare-up of her acne with small folliculitis lesions. The patient has been taking amoxicillin 500mg b.i.d. and using Tazorac cream q.d., and her face is doing well, but she has been out of her medicine now for three days. She has also been getting photofacials at Healing Waters and was wondering about what we could offer as far as cosmetic procedures and skin care products, etc. The patient is married.

FAMILY, SOCIAL, AND ALLERGY HISTORY: She has hay fever, eczema, sinus, and hives. She has no melanoma or skin cancers on psoriasis. She smokes on average one pack a day. Her mother had oral cancer. She sunburn in the past. She is on no blood thinners. Had some.

CURRENT MEDICATIONS: Lexapro, Edarbyclor, Ditropan, aspirin, vitamins.  
1.0  
The patient is a 49-year-old white female. Her weight is 221.5 pounds, blood pressure 128/74, pulse 80, and respirations 16. On examination, this is a flaring of her acne with small folliculitis bid lesions. The patient has been taking amoxicillin 500mg qid, and using Tazorac cream q1, and her face is doing well. She has been out of her medicine now for three days also. She has also been getting photofacials at Healing Waters and was wondering about what we could offer as far as cosmetic procedures and skin care products, etc. The patient is married. She is a secretary. FAMILY, SOCIAL, AND ALLERGY HISTORY: She has hay fever, eczema, sinus, and hives. She has no skin cancers or psoriasis. Her mother had oral cancer. She smokes on average one pack a day. No blood tests. Had some sunburn in the past. She is on benozoyl peroxide and aspirin, vitamins, Lexapro, Edecxor, Ditropan.

In [ ]:
read_9t = model.generate_content(['read the patient card and retell it', img9])
read_9t.resolve()
to_markdown(read_9t.text)

>  The patient is a 49-year-old white female with a history of acne. She is currently taking amoxicillin 500mg b.i.d. and using Tazorac cream 0.1% for her acne. She has been out of her medication for three days. She is also getting photofacials at Healing Waters and is wondering about what we could offer as far as cosmetic procedures and skin care products.
> 
> The patient is married and works as a secretary. She has a history of eczema, sinus, and hives. She does not have any known drug allergies. She smokes one pack of cigarettes per day. She has had some sunburn in the past. She is currently taking benzoyl peroxide and Daypro.
> 
> Her current medications include Lexapro, Effexor, Ditropan, aspirin, and vitamins.

1.5  
The patient is a 49-year-old white female, weighing 221.5 pounds. Her blood pressure is 128/74, pulse 80, and respirations 16. She presents with a flare-up of acne with small folliculitis lesions. She has been taking amoxicillin 500mg twice a day and using Tazorac cream 0.1, and her face is doing well, but she has been off her medication for three days. She has also been getting photofacials at Healing Waters and is interested in cosmetic procedures and skin care products.

She is married and works as a secretary. Her family, social, and allergy history includes hay fever, eczema, sinus, and hives. She has no history of melanoma, skin cancers, or psoriasis. Her mother had oral cancer. She smokes an average of one pack a day and has not had any blood tests. She has experienced sunburn in the past. She is currently using benzoyl peroxide and Daypro.

Her current medications are Lexapro, Effexor, Ditropan, aspirin, and vitamins.  
1.0  
The patient is a 49-year-old white female with a history of acne. She is currently taking amoxicillin 500mg b.i.d. and using Tazorac cream 0.1% for her acne. She has been out of her medication for three days. She is also getting photofacials at Healing Waters and is wondering about what we could offer as far as cosmetic procedures and skin care products.

The patient is married and works as a secretary. She has a history of eczema, sinus, and hives. She does not have any known drug allergies. She smokes one pack of cigarettes per day. She has had some sunburn in the past. She is currently taking benzoyl peroxide and Daypro.

Her current medications include Lexapro, Effexor, Ditropan, aspirin, and vitamins.

In [ ]:
read_10 = model.generate_content(['read the patient card and retell it', hw_img10])  # рукописный текст
read_10.resolve()
to_markdown(read_10.text)

>  The patient is a 40-year-old gentleman who reports swelling in his left submandibular region that occurred all of a sudden about a month and a half ago. He was originally treated with antibiotics as a possible tooth abscess. Prior to this event, a month ago, he started having night sweats. The patient reports feeling hot, when he went to bed he also reports would wake up soaked. Past medical history: HIV diagnosed in 2000. The patient also had mononucleosis at about that time. Being on anti-hepatitis viral therapy through antiviral treatment. In 2003, he had shingles. He went another small outbreak in 2009 that he treated with topical therapy. Social history: The patient is single, he currently is self employed and does antiquing. He does have a history of smoking as about a thirteen pack year history of smoking, currently smoking about a half a pack per day.

1.5  
error  
1.0  
The patient is a 40-year-old gentleman who reports swelling in his left submandibular region that occurred all of a sudden about a month and a half ago. He was originally treated with antibiotics as a possible tooth abscess. Prior to this event, a month ago, he started having night sweats. The patient reports feeling hot, when he went to bed he also reports would wake up soaked. Past medical history: HIV diagnosed in 2000. The patient also had mononucleosis at about that time. Being on anti-hepatitis viral therapy through antiviral treatment. In 2003, he had shingles. He went another small outbreak in 2009 that he treated with topical therapy. Social history: The patient is single, he currently is self employed and does antiquing. He does have a history of smoking as about a thirteen pack year history of smoking, currently smoking about a half a pack per day.

In [ ]:
read_10t = model.generate_content(['read the patient card and retell it', img10])
read_10t.resolve()
to_markdown(read_10t.text)

>  The patient is a 40-year-old gentleman who presents with a one-month history of swelling in his left submandibular region. He was initially treated with antibiotics for a possible tooth abscess. He denies any history of trauma or dental procedures. He has a history of HIV infection and was diagnosed with AIDS in 2000. He also has a history of hepatitis C and was treated with antiviral therapy in 2003. He has a history of smoking and drinks alcohol socially. He is currently unemployed and lives alone.

1.5  
The patient is a 40-year-old man who presents with swelling in his left submandibular region that started suddenly about a month and a half ago. He was initially treated with antibiotics for a possible tooth abscess. Prior to this, in March 2010, he was treated for strep throat. About a month ago, he began experiencing night sweats, feeling hot, and waking up soaked in sweat.

His past medical history is significant for an HIV diagnosis in 2000, at which time he also had mononucleosis. He reports undergoing intense anti-hepatitis viral therapy. In 2003, he had shingles and received antiviral treatment. He also mentions another small outbreak in 2009 that was treated with topical therapy.

Socially, the patient is single, self-employed, and enjoys antiquing. He has a smoking history of about 13.5 years and currently smokes about a quarter of a pack per day.  
1.0  
The patient is a 40-year-old gentleman who presents with a one-month history of swelling in his left submandibular region. He was initially treated with antibiotics for a possible tooth abscess. He denies any history of trauma or dental procedures. He has a history of HIV infection and was diagnosed with AIDS in 2000. He also has a history of hepatitis C and was treated with antiviral therapy in 2003. He has a history of smoking and drinks alcohol socially. He is currently unemployed and lives alone.

In [ ]:
read_11 = model.generate_content(['read the patient card and retell it', hw_img11])  # рукописный текст
read_11.resolve()
to_markdown(read_11.text)

>  The patient is a 17-year-old white female who complains of a four-day history of cold symptoms consisting of nasal congestion and left ear pain. She has had a dry cough, but this has not been since the first day.
> 
> In her review of systems, the patient does note some pressure in her sinuses. She denies any skin rash.
> 
> On physical examination, her vital signs are temperature 98.2, pulse is 92, respirations are 18, and blood pressure is 110/70. Her general appearance is that of a healthy appearing white female who appears only mildly ill. HEENT: Pupils are round, atraumatic. No history of allergies. Head and neck are free from diseases such as allergic reactions. This is the fourth time in a year that she has had symptoms suggestive of tonsillitis, pharyngitis, and sinusitis. She denies edema, exudate, erythema, tenderness, or ulceration. Sinuses do show some mild anterior/posterior cervical lymph nodes. There is no nuchal rigidity and no meningismus. Her neck has full range of motion without pain. Lungs are clear to auscultation bilaterally without rales, rhonchi, or wheezes. Heart rate and rhythm are regular without murmurs, rubs, or gallops.

1.5  
The patient is a 17-year-old white female who is complaining of a four-day history of cold symptoms consisting of nasal congestion and left ear pain. She has had a dry cough, but this has not been since the first day.

REVIEW OF SYSTEMS: The patient does note some pressure in her sinuses. She denies any skin rash.

PHYSICAL EXAMINATION/ VITAL SIGNS: Temp is 36.9, pulse is elevated at 92, other vital signs are all within normal limits.

GENERAL: Patient is a healthy-appearing white female, appears only mildly ill. HEENT: Head is normocephalic, atraumatic. No history of allergic reactions. Suffers from diseases such as pharyngitis and sinusitis for the fourth time in a year. Pharynx shows no erythema, tonsillar edema, or exudate. Sinuses do show some mild tenderness to percussion. NECK: No meningismus or enlarged anterior/posterior cervical lymph nodes. HEART: Regular rate and rhythm without murmurs, rubs, or gallops. LUNGS: Clear without rales, rhonchi, or wheezes.   
1.0  
The patient is a 17-year-old white female who complains of a four-day history of cold symptoms consisting of nasal congestion and left ear pain. She has had a dry cough, but this has not been since the first day.

In her review of systems, the patient does note some pressure in her sinuses. She denies any skin rash.

On physical examination, her vital signs are temperature 98.2, pulse is 92, respirations are 18, and blood pressure is 110/70. Her general appearance is that of a healthy appearing white female who appears only mildly ill. HEENT: Pupils are round, atraumatic. No history of allergies. Head and neck are free from diseases such as allergic reactions. This is the fourth time in a year that she has had symptoms suggestive of tonsillitis, pharyngitis, and sinusitis. She denies edema, exudate, erythema, tenderness, or ulceration. Sinuses do show some mild anterior/posterior cervical lymph nodes. There is no nuchal rigidity and no meningismus. Her neck has full range of motion without pain. Lungs are clear to auscultation bilaterally without rales, rhonchi, or wheezes. Heart rate and rhythm are regular without murmurs, rubs, or gallops.

In [ ]:
read_11t = model.generate_content(['read the patient card and retell it', img11])
read_11t.resolve()
to_markdown(read_11ty.text)

> The patient is a 17-year-old white female presenting with a four-day history of cold symptoms including nasal congestion and left ear pain. She initially had a dry cough, which has since resolved. She reports pressure in her sinuses but denies any skin rash. 
> 
> Her temperature is 36.9 degrees Celsius, and her pulse is elevated at 92 bpm. All other vital signs are within normal limits. 
> 
> She appears healthy and only mildly ill. Her head is normocephalic and atraumatic, with no history of allergic reactions. She has a history of frequent pharyngitis and sinusitis, this being the fourth occurrence this year. 
> 
> Examination reveals no erythema, tonsillar edema, or exudate in the pharynx. Sinuses are mildly tender to percussion. There is no meningismus or enlarged anterior/posterior cervical lymph nodes. 
> 
> Her heart rate and rhythm are regular, with no murmurs, rubs, or gallops. Lungs are clear without rales, rhonchi, or wheezes. 


1.5  
The patient is a 17-year-old white female presenting with a four-day history of cold symptoms including nasal congestion and left ear pain. She initially had a dry cough, which has since resolved. She reports pressure in her sinuses but denies any skin rash.

Her temperature is 36.9 degrees Celsius, and her pulse is elevated at 92 bpm. All other vital signs are within normal limits.

She appears healthy and only mildly ill. Her head is normocephalic and atraumatic, with no history of allergic reactions. She has a history of frequent pharyngitis and sinusitis, this being the fourth occurrence this year.

Examination reveals no erythema, tonsillar edema, or exudate in the pharynx. Sinuses are mildly tender to percussion. There is no meningismus or enlarged anterior/posterior cervical lymph nodes.

Her heart rate and rhythm are regular, with no murmurs, rubs, or gallops. Lungs are clear without rales, rhonchi, or wheezes.  
1.0  
The patient is a 17-year-old white female presenting with a four-day history of cold symptoms including nasal congestion and left ear pain. She initially had a dry cough, which has since resolved. She reports pressure in her sinuses but denies any skin rash.

Her temperature is 36.9 degrees Celsius, and her pulse is elevated at 92 bpm. All other vital signs are within normal limits.

She appears healthy and only mildly ill. Her head is normocephalic and atraumatic, with no history of allergic reactions. She has a history of frequent pharyngitis and sinusitis, this being the fourth occurrence this year.

Examination reveals no erythema, tonsillar edema, or exudate in the pharynx. Sinuses are mildly tender to percussion. There is no meningismus or enlarged anterior/posterior cervical lymph nodes.

Her heart rate and rhythm are regular, with no murmurs, rubs, or gallops. Lungs are clear without rales, rhonchi, or wheezes.

In [ ]:
read_12 = model.generate_content(['read the patient card and retell it', hw_img12])   # рукописный текст
read_12.resolve()
to_markdown(read_12.text)

>  The patient is a 61-year-old gentleman who presents for further evaluation of feet and hand cramps. He describes that the foot cramps are much more notable than the hand ones. He reports that he develops muscle contractions of his toes on both feet. These occur exclusively at night. He has a history of HIV for 21 years. He was taking antiretroviral medications, but stopped about six or seven years ago. He reports that he was unable to tolerate the medications due to serve stomach upset. He has no known drug allergies. He lives alone. He denies alcohol, tobacco, or illicit drug use. He is now retired. He is very active and walks about four miles every few days.

1.5  
The patient is a 61-year-old gentleman who presents for further evaluation of feet and hand cramps. He describes that the foot cramps are much more notable than the hand ones. He reports that he develops muscle contractions of his toes on both feet. These occur exclusively at night. He has a history of HIV for 21 years. He was taking antiretroviral medications, but stopped about six or seven years ago. He reports that he was unable to tolerate the medications due to severe stomach upset.

ALLERGIES: He has no known drug allergies.

Social History: He lives alone. He denies alcohol, tobacco, or illicit drug use. He is now retired. He is very active and walks about four miles every few days.  
1.0  
The patient is a 61-year-old gentleman who presents for further evaluation of feet and hand cramps. He describes that the foot cramps are much more notable than the hand ones. He reports that he develops muscle contractions of his toes on both feet. These occur exclusively at night. He has a history of HIV for 21 years. He was taking antiretroviral medications, but stopped about six or seven years ago. He reports that he was unable to tolerate the medications due to serve stomach upset. He has no known drug allergies. He lives alone. He denies alcohol, tobacco, or illicit drug use. He is now retired. He is very active and walks about four miles every few days.

In [ ]:
read_12t = model.generate_content(['read the patient card and retell it', img12])
read_12t.resolve()
to_markdown(read_12t.text)

>  The patient is a 61-year-old gentleman who presents for further evaluation of foot and hand cramps. He describes the foot cramps are much more notable than the hand ones. He reports that he develops muscle contractions of his toes on both feet. These occur exclusively at night. He has a history of HIV for 21 years. He was taking antiretroviral medications, but stopped about six or seven years ago. He reports that he was unable to tolerate the medications due to severe stomach upset. He has no known drug allergies. He lives alone. He denies alcohol, tobacco, or illicit drug use. He is now retired. He is very active and walks about four miles every few days.

1.5  
The patient is a 61-year-old man who is seeking further evaluation for foot and hand cramps. He experiences foot cramps more severely than hand cramps. He reports muscle contractions in his toes on both feet, occurring exclusively at night. He has a 21-year history of HIV and was previously on antiretroviral medications but stopped approximately six or seven years ago due to severe stomach upset. He has no known drug allergies. He lives alone, denies alcohol, tobacco, or illicit drug use, and is currently retired. He maintains an active lifestyle, walking about four miles every few days  
1.0  
The patient is a 61-year-old gentleman who presents for further evaluation of foot and hand cramps. He describes the foot cramps are much more notable than the hand ones. He reports that he develops muscle contractions of his toes on both feet. These occur exclusively at night. He has a history of HIV for 21 years. He was taking antiretroviral medications, but stopped about six or seven years ago. He reports that he was unable to tolerate the medications due to severe stomach upset. He has no known drug allergies. He lives alone. He denies alcohol, tobacco, or illicit drug use. He is now retired. He is very active and walks about four miles every few days.

## test1 (True=NO)    
Вручную на примере первого критеия и первого пациента посмотрим как работает модель Gemini 1.0
ttf - text-text-photo  
tff - text-photo-photo  
ttw - text-text-hand writing  
tfw -text-photo-hand writing  
0 - zero_shot  
2 - few_shot  
cot  
tot

In [ ]:
response1_ttf0 = model.generate_content([prompt_zero_shot, text_crit1, img1], stream=True)
response1_ttf0.resolve()
to_markdown(response1_ttf0.text)

In [ ]:
response1_ttf2 = model.generate_content([prompt_few_shot, text_crit1, img1], stream=True)
response1_ttf2.resolve()
to_markdown(response1_ttf2.text)

# """Inclusion Criteria: Subjects must have smoked on average of at least ten cigarettes per day during the past year; - Yes
# Patient weighs more than 220 pounds. - Yes
# Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease; - No
# Uncontrolled hypertension. - No)"""

In [ ]:
response1_ttfC = model.generate_content([prompt_CoT, text_crit1, img1], stream=True)
response1_ttfC.resolve()
to_markdown(response1_ttfC.text)
# """Inclusion Criteria: Subjects must have smoked on average of at least ten cigarettes per day during the past year; Yes, the patient has smoked an average of 15 cigarettes per day for the past 20 years.
# Patient weighs more than 220 pounds. Yes, the patient weighs 275 pounds.
# Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease; It is not clear from the information provided whether the patient has any history of clinically significant cardiovascular disease.
# Uncontrolled hypertension. The patient's blood pressure is 160-165/100-110 mm Hg. It is not clear from the information provided whether the patient's hypertension is controlled.
# Based on the information provided, it is not possible to determine whether the patient meets the criteria for this study. Further information is needed to determine the patient's history of cardiovascular disease and whether their hypertension is controlled."""

In [ ]:
response1_ttfT = model.generate_content([prompt_ToT, text_crit1, img1], stream=True)
response1_ttfT.resolve()
to_markdown(response1_ttfT.text)
# """Expert 1: First, the patient must have smoked at least 10 cigarettes per day during the past year. The patient has smoked an average of 15 cigarettes per day for the past 20 years, so this criterion is met.
# Expert 2: Second, the patient must weigh more than 220 pounds. The patient weighs 275 pounds, so this criterion is also met.
# Expert 3: Third, the patient must not have any history of clinically significant cardiovascular disease. The patient has a history of asthma, but this is not a cardiovascular disease, so this criterion is also met.
# Expert 1: Finally, the patient must not have uncontrolled hypertension. The patient's blood pressure is 160/100 mm Hg, which is slightly elevated, but it is not uncontrolled. So, this criterion is also met.
# Overall, the patient meets all of the inclusion criteria and none of the exclusion criteria, so they are eligible for the study."""

In [ ]:
response1_tff0 = model.generate_content([prompt_zero_shot, crit1, img1], stream=True)
response1_tff0.resolve()
to_markdown(response1_tff0.text)

In [ ]:
response1_tff2 = model.generate_content([prompt_few_shot, crit1, img1], stream=True)
response1_tff2.resolve()
to_markdown(response1_tff2.text)
# """Inclusion Criteria:

# Subjects must have smoked on average of at least ten cigarettes per day during the past year: Yes
# Patient weighs more than 220 pounds: Yes
# Exclusion Criteria:

# Subjects with any history of clinically significant cardiovascular disease: No
# Uncontrolled hypertension: No
# The patient meets all the inclusion criteria and none of the exclusion criteria, so he can be included in the study."""

In [ ]:
response1_ttfC = model.generate_content([prompt_CoT, crit1, img1], stream=True)
response1_ttfC.resolve()
to_markdown(response1_ttfC.text)
# """This patient meets the inclusion criteria.
# The patient has smoked an average of 15 cigarettes per day for the past 20 years and weighs 275 lbs.
# This patient does not meet the exclusion criteria. The patient has a history of asthma but it is controlled with medication.
# The patient's blood pressure is slightly elevated but it is not uncontrolled hypertension."""

In [ ]:
response1_ttfT = model.generate_content([prompt_ToT, crit1, img1], stream=True)
response1_ttfT.resolve()
to_markdown(response1_ttfT.text)
# """Expert 1: First, I would check if the patient meets the inclusion criteria. The patient has smoked an average of 15 cigarettes per day for the past 20 years, which meets the first criterion. The patient weighs 275 lbs, which meets the second criterion. Therefore, the patient meets the inclusion criteria.
# Expert 2: I agree with Expert 1. The patient meets the inclusion criteria.
# Expert 3: I agree with Experts 1 and 2. The patient meets the inclusion criteria.
# Expert 1: Next, I would check if the patient meets the exclusion criteria. The patient has a history of asthma, which is a clinically significant cardiovascular disease. Therefore, the patient meets the exclusion criteria.
# Expert 2: I agree with Expert 1. The patient meets the exclusion criteria.
# Expert 3: I agree with Experts 1 and 2. The patient meets the exclusion criteria.
# Overall, the patient does not meet the patient selection criteria for this medical study."""

In [ ]:
response1_ttw0 = model.generate_content([prompt_zero_shot, text_crit1, hw_img1], stream=True)
response1_ttw0.resolve()
to_markdown(response1_ttw0.text)

In [ ]:
response1_ttw2 = model.generate_content([prompt_few_shot, text_crit1, hw_img1], stream=True)
response1_ttw2.resolve()
to_markdown(response1_ttw2.text)
# """Inclusion Criteria: Subjects must have smoked on average of at least ten cigarettes per day during the past year; - Yes Patient weighs more than 220 pounds. - none
# Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease; - none Uncontrolled hypertension. - none"""

In [ ]:
response1_ttwC = model.generate_content([prompt_CoT, text_crit1, hw_img1], stream=True)
response1_ttwC.resolve()
to_markdown(response1_ttwC.text)
# """Inclusion Criteria: Subjects must have smoked on average of at least ten cigarettes per day during the past year; => Yes, the patient smokes a pack a day Patient weighs more than 220 pounds. => Unknown
# Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease; => Unknown Uncontrolled hypertension. => Unknown
# The patient meets 1 out of 2 inclusion criteria and has 2 unknown exclusion criteria. Therefore, we cannot determine if the patient is eligible for this study based on the information provided."""


In [ ]:
response1_ttwT = model.generate_content([prompt_ToT, text_crit1, hw_img1], stream=True)
response1_ttwT.resolve()
to_markdown(response1_ttwT.text)
# """Expert 1: First, the patient smokes a pack of cigarettes a day, so they meet the inclusion criteria.
# Expert 2: The patient weighs 220 pounds, so they meet the inclusion criteria. Expert 3: The patient has a history of tonsillitis, but no history of cardiovascular disease or hypertension, so they meet the inclusion criteria.
# All experts agree that the patient meets the inclusion criteria and should be included in the study."""

In [ ]:
response1_tfw0 = model.generate_content([prompt_zero_shot, crit1, hw_img1], stream=True)
response1_tfw0.resolve()
to_markdown(response1_tfw0.text)

In [ ]:
response1_tfw2 = model.generate_content([prompt_few_shot, crit1, hw_img1], stream=True)
response1_tfw2.resolve()
to_markdown(response1_tfw2.text)
# """Inclusion Criteria:

# Subjects must have smoked on average of at least ten cigarettes per day during the past year: Yes
# Patient weighs more than 220 pounds: No
# Exclusion Criteria:

# Subjects with any history of clinically significant cardiovascular disease: No
# Uncontrolled hypertension: Yes"""

In [ ]:
response1_tfwC = model.generate_content([prompt_CoT, crit1, hw_img1], stream=True)
response1_tfwC.resolve()
to_markdown(response1_tfwC.text)
# """Inclusion Criteria:

# Subjects must have smoked on average of at least ten cigarettes per day during the past year - Yes, the patient smokes 1 pack a day which is 20 cigarettes a day.
# Patient weighs more than 220 pounds - No, the patient weighs 150 pounds.
# Exclusion Criteria:

# Subjects with any history of clinically significant cardiovascular disease - No, the patient denies any history of cardiovascular disease.
# Uncontrolled hypertension - No, the patient's blood pressure is 120/80 mmHg.
# The patient does not meet all of the inclusion criteria and does not have any of the exclusion criteria, so they are not eligible for the study."""

In [ ]:
response1_tfwT = model.generate_content([prompt_ToT, crit1, hw_img1], stream=True)
response1_tfwT.resolve()
to_markdown(response1_tfwT.text)
# """Expert 1: First, I would check if the patient smokes on average at least 10 cigarettes per day.
# Expert 2: I agree. That's the first thing to check.
# Expert 3: Yes, and the patient smokes a pack a day, so that's more than 10 cigarettes per day.
# Expert 1: Next, I would check if the patient weighs more than 220 pounds.
# Expert 2: Me too.
# Expert 3: The patient weighs 250 pounds, so that's more than 220 pounds.
# Expert 1: Finally, I would check if the patient has any history of clinically significant cardiovascular disease or uncontrolled hypertension.
# Expert 2: I agree.
# Expert 3: The patient denies any history of cardiovascular disease or hypertension, so that's good.
# Expert 1: So, the patient meets all of the inclusion criteria and none of the exclusion criteria. Therefore, the patient is eligible for the study.
# Expert 2: I agree.
# Expert 3: Me too."""

## test 2. В цикле обработаем все данные. Заранее настроим параметры конфиденциальности модели.   
Для модели Gemini 1.5 стоит ограничение на количество запросов в минуту, поэтому для корректной работы пришлось добавлять time.sleep().

In [ ]:
# Создаем пустой список для хранения результатов                                             # prompt_zero_shot, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response_ttfC = model.generate_content([prompt_zero_shot, crit, img], stream=False)
        response_ttfC.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        results.append({'Response': response_ttfC.text, 'img': f'test{i}.png', 'crit': crit})

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_zero_shot_1.5.xlsx', index=False)

In [ ]:
# Создаем пустой список для хранения результатов                                          # prompt_few_shot, crit (текст и фото), img   - фото печат. карта пациента
results = []
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_few_shot, crit, img], stream=False,
                                          safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)


# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_few_shot_1.5.xlsx', index=False)


Processing criteria:  33%|███▎      | 2/6 [03:54<07:46, 116.72s/it]
FinishReason.SAFETY
[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: MEDIUM
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]

FinishReason.SAFETY
[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: MEDIUM
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]

FinishReason.SAFETY
[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]
Processing criteria: 100%|██████████| 6/6 [14:05<00:00, 140.96s/it]

In [ ]:
# Создаем пустой список для хранения результатов                                                             # prompt_ToT, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_ToT, crit, img], stream=False,
                                               safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})

        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_ToT_1.5.xlsx', index=False)

In [ ]:
# Создаем пустой список для хранения результатов                                                               # prompt_CoT, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_CoT, crit, img], stream=True, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря

        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_CoT_1.5.xlsx', index=False)

In [ ]:

# Создаем пустой список для хранения результатов                                                                 # prompt_zero_shot, crit (текст и фото), hw_img   - фото рукописная карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response_ttfC = model.generate_content([prompt_zero_shot, crit, img], stream=True)
        response_ttfC.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        results.append({'Response': response_ttfC.text, 'img': f'test{i}.png', 'crit': crit})

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_zero_shot_1.5.xlsx', index=False)

In [ ]:
# Создаем пустой список для хранения результатов
results = []
from tqdm import tqdm
# Пробегаемся по всем критериям
#for crit in [text_crit1, text_crit2, text_crit3, crit1, crit2, crit3]:
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_few_shot, crit, img], stream=True, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_few_shot_1.5.xlsx', index=False)


In [ ]:
# Создаем пустой список для хранения результатов
results = []

# Пробегаемся по всем критериям
#for crit in [text_crit1, text_crit2, text_crit3, crit1, crit2, crit3]:
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_ToT, crit, img], stream=True, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_ToT_1.5.xlsx', index=False)

In [ ]:
# Создаем пустой список для хранения результатов
results = []

# Пробегаемся по всем критериям
#for crit in [text_crit1, text_crit2, text_crit3, crit1, crit2, crit3]:
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_CoT, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_CoT_1.5.xlsx', index=False)

In [ ]:
import time

# Создаем пустой список для хранения результатов
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_ToT, crit, img], stream=True, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        # Ждем 10 секунд перед следующим запросом
        time.sleep(10)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_ToT_1.5.xlsx', index=False)

In [ ]:
# Создаем пустой список для хранения результатов                                                             # prompt_ToT, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_ToT, crit, img], stream=False,
                                               safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})

        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        # Ждем __ секунд перед следующим запросом
        time.sleep(15)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_ToT_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [27:55<00:00, 279.26s/it]


In [ ]:
#Создаем пустой список для хранения результатов
results = []

# Пробегаемся по всем критериям
#for crit in [text_crit1, text_crit2, text_crit3, crit1, crit2, crit3]:
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_CoT, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(15)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_CoT_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [32:47<00:00, 327.96s/it]


In [ ]:
# Создаем пустой список для хранения результатов                                                               # prompt_CoT, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_CoT, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря

        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(15)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_CoT_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [26:03<00:00, 260.58s/it]


In [ ]:
# Создаем пустой список для хранения результатов                                          # prompt_few_shot, crit (текст и фото), img   - фото печат. карта пациента
results = []
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_few_shot, crit, img], stream=False,
                                          safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
                          })
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(20)


# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_few_shot_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [29:35<00:00, 295.96s/it]


In [ ]:
# Создаем пустой список для хранения результатов
results = []
from tqdm import tqdm
# Пробегаемся по всем критериям
#for crit in [text_crit1, text_crit2, text_crit3, crit1, crit2, crit3]:
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_few_shot, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря
        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(20)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_few_shot_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [35:57<00:00, 359.63s/it]


In [ ]:
# Создаем пустой список для хранения результатов                                             # prompt_zero_shot, crit (текст и фото), img   - фото печат. карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/text/test{i}.png')

        # Генерируем контент
        response = model.generate_content([prompt_zero_shot, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря

        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(20)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/results_prompt_zero_shot_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [26:35<00:00, 265.95s/it]


In [ ]:
# Создаем пустой список для хранения результатов                                                                 # prompt_zero_shot, crit (текст и фото), hw_img   - фото рукописная карта пациента
results = []

# Пробегаемся по всем критериям
for crit in tqdm([text_crit1, text_crit2, text_crit3, crit1, crit2, crit3], desc="Processing criteria"):
    for i in range(1, 12):
        # Загружаем изображение
        img = PIL.Image.open(f'/content/drive/MyDrive/диплом/test/photo/photo{i}.jpg')

        # Генерируем контент
        response = model.generate_content([prompt_zero_shot, crit, img], stream=False, safety_settings={
                          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE})
        response.resolve()

        # Добавляем результат, название изображения и критерий в список в виде словаря

        try: results.append({'Response': response.text, 'img': f'test{i}.png', 'crit': crit})
        except ValueError:
        # If the response doesn't contain text, check if the prompt was blocked.
              print(response.prompt_feedback)
        # Also check the finish reason to see if the response was blocked.
              print(response.candidates[0].finish_reason)
        # If the finish reason was SAFETY, the safety ratings have more details.
              print(response.candidates[0].safety_ratings)

        time.sleep(15)

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Сохраняем результаты в файл Excel
df.to_excel('/content/drive/MyDrive/диплом/test/hw_prompt_zero_shot_1.5.xlsx', index=False)

Processing criteria: 100%|██████████| 6/6 [34:33<00:00, 345.51s/it]


### test with Lang Chain

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", generation_config={"temperature": 0.0})

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": '''This is the patient selection criterion for a medical study.
Next, I will show you patient cards and ask if this patient fits the criteria for this study,
and you should answer 'yes' or 'no'
Answer step by step.
Clinical Trial Criteria:{
Inclusion Criteria:
Subjects must have smoked on average of at least ten cigarettes per day during the past year;
Patient weighs more than 220 pounds.
Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease;
Uncontrolled hypertension
}''',
        },
        {"type": "image_url", "image_url": img1},
    ]
)
llm.invoke([message])

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", generation_config={"temperature": 0.0})  # langchain с историей диалога (не работает с Img)
memory = ConversationBufferMemory()
conv_buf_mem = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True #отображать логи в консоли или нет
)

# "ответ": "

# Выкуривает ли пациент в среднем не менее десяти сигарет в день в течение последнего года?Да, пациент выкуривал в среднем 15 сигарет в день в течение последних 20 лет.
# Весит ли пациент более 220 фунтов?Да, пациент весит 275 фунтов.
# Есть ли у пациента в анамнезе клинически значимые сердечно-сосудистые заболевания?\N В предоставленной информации не уточняется, были ли у пациента в анамнезе клинически значимые сердечно-сосудистые заболевания.
# Есть ли у пациента неконтролируемая артериальная гипертензия?\В предоставленной информации не уточняется, есть ли у пациента неконтролируемая артериальная гипертензия.
#  Однако АД пациента составляет систолическое 140-150 мм рт.ст., диастолическое 90-100 мм рт.ст., что считается артериальной гипертензией 1 стадии.
#  Основываясь на предоставленной информации, пациент соответствует критериям включения в исследование (выкуривает в среднем не менее десяти сигарет в день в течение последнего года и весит более 220 фунтов).
#  Однако неясно, соответствует ли пациент критериям исключения (имеет ли в анамнезе клинически значимые сердечно-сосудистые заболевания или неконтролируемую артериальную гипертензию)".

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", generation_config={"temperature": 0.0}) # используя langchain (img)
message2 = HumanMessage(
    content=[
        {
            "type": "text",
            "text": '''This is the patient selection criterion for a medical study.
Next, I will show you patient cards and ask if this patient fits the criteria for this study,
and you should answer 'yes' or 'no'
Answer step by step.
Clinical Trial Criteria:{
Inclusion Criteria:
Subjects must have smoked on average of at least ten cigarettes per day during the past year;
Patient weighs more than 220 pounds.
Exclusion Criteria: Subjects with any history of clinically significant cardiovascular disease;
Uncontrolled hypertension
}''',
        },
        {"type": "image_url", "image_url": img2},
    ]
)
llm.invoke([message2])